# Load in dataset

In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

In [7]:
data = pd.read_csv("Datasets.csv")

In [16]:
parameters = list(data.columns)
cell_data = data.filter(items=parameters[:-1])

# Image generation

In [33]:
from math import sin, cos, radians

In [151]:
"""
Takes in a Pandas series and generates an image of the re-entrant auxetic
    described by the parameters

Arguments:
* data: Pandas series with 'Slant cell length', 'Cell thickness', 'Cell angle', 
    'Vertical cell length'
* show: False if plot should not be displayed
* save: True if the plot should be saved
* fname: name of the file to save the plot into if save=True
"""
def gen_image(data, show=True, save=False, fname=''):
    # ignoring cell thickness for now
    slant_len = data["Slant cell length"]
    thickness = data["Cell thickness"]
    angle = data["Cell angle"]
    height = data["Vertical cell length"]

    # for calculating x and y coordinates of plot lines
    slant_dx = slant_len * cos(radians(90 - angle))
    slant_dy = slant_len * sin(radians(90 - angle))

    # plot point generation
    top = 1 + height
    bottom = 1
    center = 1 + slant_dx
    left = 1
    right = 1 + 2*slant_dx
    # [ [[x values of 1st slant], [y values of 1st slant]], [...], [...], [...] ]
    slants = [
        [[left, center], [bottom, bottom + slant_dy]],
        [[center, right], [bottom + slant_dy, bottom]],
        [[left, center], [top, top - slant_dy]],
        [[center, right],[top - slant_dy, top]]
    ]
    # vertical lines
    verticals = [
        [[left, left], [bottom, top]],
        [[center, center], [bottom + slant_dy, bottom - 0.2*height]],
        [[right, right], [bottom, top]],
        [[center, center], [top - slant_dy, top + 0.2*height]]
    ]

    # plotting and saving
    plt.cla()
    for slant in slants:
        plt.plot(*slant, color='black', linewidth=thickness*0.5)
    for line in verticals:
        plt.plot(*line, color='black', linewidth=thickness*0.5)
    plt.axis('off')
    
    if save:
        plt.savefig(fname)
    if not show:
        plt.close()

"""
Saves a group of images

Arguments:
* data: Pandas dataframe
* indices: list of indices of rows in the dataframe to
    generate images from
"""
def save_images(data, indices, folder='test'):
    excluded = []
    for idx in indices:
        if idx >= data.shape[0]:
            excluded.append(idx)
            continue
        gen_image(data.iloc[idx], show=False, save=True, fname=f'./images/{folder}/{idx}.png')
    if excluded:
        print(f"indices excluded (not in range): {excluded}")

## Train vs. Test Split

In [117]:
import random

In [ ]:
train = data.sample(n=1500)
test = data.sample(n=2500)
print(f'{train.shape[0]} training samples')
print(f'{test.shape[0]} test samples')

Checking distribution of each parameter

In [ ]:
# NOT NECESSARY FOR MODEL
# original data
for param in parameters[:-1]:
    values = list(data[param])
    unique_vals = set(values)
    plt.title(f"Original: {param}")
    plt.hist(values, bins=len(unique_vals), edgecolor='black')
    plt.show()

In [ ]:
# NOT NECESSARY FOR MODEL
# train data
for param in parameters[:-1]:
    values = list(train[param])
    unique_vals = set(values)
    plt.title(f"Train: {param}")
    plt.hist(values, bins=len(unique_vals), edgecolor='black')
    plt.show()

In [ ]:
# NOT NECESSARY FOR MODEL
# test data
for param in parameters[:-1]:
    values = list(test[param])
    unique_vals = set(values)
    plt.title(f"Test: {param}")
    plt.hist(values, bins=len(unique_vals), edgecolor='black')
    plt.show()

Generate ground truth labels and save images

In [213]:
import torch
import torch.nn as nn
from torch import tensor, optim

from torch.utils.data import Dataset, DataLoader

from torchvision import transforms

import PIL
from PIL import Image


In [ ]:
transform = transforms.Compose([transforms.ToTensor()])
# Image.open("./images/test/4.png").convert("RGB")


In [ ]:
train_indices = list(train.index)
test_indices = list(test.index)

In [ ]:
# Train labels -- may not need
train_labels = []
for idx in train_indices:
    poisson_ratio = data.iloc[idx]["Poisson's ratio"]
    train_labels.append(poisson_ratio)

In [ ]:
# Test labels -- may not need
test_labels = []
for idx in test_indices:
    poisson_ratio = data.iloc[idx]["Poisson's ratio"]
    test_labels.append(poisson_ratio)

In [ ]:
# Save training images -- takes ~90 seconds to run
save_images(data, train_indices, folder='train')

In [ ]:
# Save test images -- takes ~135 seconds to run
save_images(data, test_indices, folder='test')

In [229]:
# takes ~1 minute

train_images = []
for idx in train_indices:
    img = Image.open(f"./images/train/{idx}.png").convert("RGB")
    train_images.append((transform(img), data.iloc[idx]["Vertical cell length"], data.iloc[idx]["Poisson's ratio"]))

test_images = []
for idx in test_indices:
    img = Image.open(f"./images/test/{idx}.png").convert("RGB")
    test_images.append((transform(img), data.iloc[idx]["Vertical cell length"], data.iloc[idx]["Poisson's ratio"]))

# Model training

In [225]:
from tqdm.notebook import tqdm

Adapted from [Medium article](https://medium.com/@iamarjunchandra/mixed-input-data-in-pytorch-cnn-mlp-8aeff336e8a3) about mixed input data

In [230]:
device = torch.device("cpu")

# HYPERPARAMETERS
LATENT_DIM = 256
BATCH_SIZE = 32
LEARNING_RATE = 1e-3
EPOCHS = 50

# DATA LOADERS
train_image_loader = DataLoader(train_images, batch_size=BATCH_SIZE, shuffle=True)
test_image_loader = DataLoader(test_images, batch_size=BATCH_SIZE, shuffle=False)

# NEURAL NETWORK
class NeuralNetwork(nn.Module):
    def __init__(self, latent_dim=LATENT_DIM):
        super(NeuralNetwork, self).__init__()
        self.image_features_ = nn.Sequential(
            nn.Conv2d(3, 16, kernel_size=5, stride=2, padding=2),
            nn.ReLU(inplace=True),
            nn.MaxPool2d(kernel_size=3, stride=2),
            nn.Dropout(),
            nn.Conv2d(16, 64, kernel_size=5, padding=2),
            nn.ReLU(inplace=True),
            nn.MaxPool2d(kernel_size=3, stride=2),
            nn.Dropout()
        )
        self.numeric_features_ = nn.Sequential(
            nn.Linear(9, 64),
            nn.ReLU(inplace=True),
            nn.Dropout(),
            nn.Linear(64, 64*64),
            nn.ReLU(inplace=True),
            nn.Dropout()
        )
        self.combined_features_ = nn.Sequential(
            nn.Linear(64*64*2, 64*64*2*2),
            nn.ReLU(inplace=True),
            nn.Dropout(),
            nn.Linear(64*64*2*2, 64*64*2),
            nn.ReLU(inplace=True),
            nn.Linear(64*3*3*2, 64),
            nn.Linear(64, 1)
        )
    def forward(self, x, y):
        x = self.image_features_(x)
        x = x.view(-1, 64*64)
        y = self.numeric_features_(y)
        z = torch.cat((x,y), 1)
        z = self.combined_features_(z)
        return z

model = NeuralNetwork().to(device)
criterion = nn.MSELoss()
optimizer = optim.Adam(model.parameters(), lr=LEARNING_RATE)

In [231]:
# training loop
def train_nn(model, train_image_loader, epochs=EPOCHS):
    model.train()
    loss_history = []

    for epoch in tqdm(range(epochs), desc="Training Progress"):
        total_loss = 0

        for images, heights, poisson in train_image_loader:
            images = images.to(device)

            optimizer.zero_grad()
            poisson_pred = model(images, heights)
            loss = criterion(poisson_pred, poisson)
            loss.backward
            optimizer.step()

            total_loss += loss.item()
        
        avg_loss = total_loss / len(train_image_loader)
        loss_history.append(avg_loss)
        if epoch % 10 == 0:
            print(f"Epoch [{epoch+1}/epochs], Loss: {avg_loss:.4f}")

    # plot loss history
    plt.plot(range(1, epochs + 1), loss_history, marker='o', linestyle='-')
    plt.xlabel("epoch")
    plt.ylabel("Loss")
    plt.title("Training Loss Over Time")
    plt.grid()
    plt.show()

    return loss_history

loss_history = train_nn(model, train_image_loader, epochs=EPOCHS)

Training Progress:   0%|          | 0/50 [00:00<?, ?it/s]

RuntimeError: shape '[-1, 4096]' is invalid for input of size 9545728

In [ ]:
# copied from article, to be adapted

for epoch in range(1, n_epochs + 1):
    loss_train = 0
    for imgs, numeric_features, price in train_loader:
        imgs = imgs.to(device)
        numeric_features = numeric_features.to(device)
        price = price.to(device)
        output=model(imgs, numeric_features)

        loss = loss_fn(output, price)

        # L2 Regularization
        l2_lambda = 0.001
        l2_norm = sum(p.pow(2).sum() for p in model.parameters())
        loss = loss + l2_lambda * l2_norm

        optimizer.zero_grad()
        loss.backward()
        optimizer.step()

        loss_train += loss.item()